In [281]:
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime

In [282]:
path = './db/課題.sqlite'
con = sqlite3.connect(path)
cur = con.cursor()
sql_select_weather = "SELECT * FROM weather"
cur.execute(sql_select_weather)
data = cur.fetchall()
con.close()


In [283]:
# いらない特徴量を排除
df = pd.DataFrame(data, columns=["気圧", "海面気圧", "降水量", "1時間降水", "10分降水", "平均気温", "最高気温", "最低気温",
    "平均湿度", "最小湿度", "平均風速", "最大風速", "風向", "最大瞬間風速", "最大瞬間風向", "日照時間", "降雪", "積雪", "昼", "夜", "月", "日"])

# df.dtypes
exclude = ['風向', '最大瞬間風向', '昼', '夜']
object_columns = [col for col in df.columns if col not in exclude]
# 型を整える
for column in object_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# datetimeに直す
def create_datetime(row):
    return datetime(2023, int(row['月']), int(row['日']))
df["年月日"] = df.apply(create_datetime, axis=1)
df = df[["気圧", "降水量", "平均気温", "最高気温", "最低気温", "平均湿度", "年月日"]]
df

,気圧,降水量,平均気温,最高気温,最低気温,平均湿度,年月日
0,1013.5,NaN,7.4,12.1,2.3,59.0,2023-01-01
1,1013.4,0.0,6.9,11.1,4.3,51.0,2023-01-02
2,1012.8,NaN,6.5,10.9,3.0,45.0,2023-01-03
3,1012.8,NaN,5.9,10.5,2.1,48.0,2023-01-04
4,1016.4,NaN,6.2,11.0,2.9,36.0,2023-01-05
...,...,...,...,...,...,...,...
4375,1020.2,NaN,8.8,13.3,5.2,52.0,2023-12-27
4376,1020.1,NaN,8.1,11.4,6.4,53.0,2023-12-28
4377,1016.0,NaN,8.8,13.4,4.1,59.0,2023-12-29
4378,1014.9,NaN,9.8,14.6,5.9,60.0,2023-12-30


In [284]:
# NaNと雨の日を排除
sun_day = df[df["降水量"].isna()].drop("降水量", axis=1)
sun_day = sun_day.dropna(subset=["最高気温", "最低気温"])
# sun_day.isna().sum()
sun_day

,気圧,平均気温,最高気温,最低気温,平均湿度,年月日
0,1013.5,7.4,12.1,2.3,59.0,2023-01-01
2,1012.8,6.5,10.9,3.0,45.0,2023-01-03
3,1012.8,5.9,10.5,2.1,48.0,2023-01-04
4,1016.4,6.2,11.0,2.9,36.0,2023-01-05
5,1014.6,6.1,10.0,1.8,46.0,2023-01-06
...,...,...,...,...,...,...
4374,1017.0,8.4,13.4,3.7,56.0,2023-12-26
4375,1020.2,8.8,13.3,5.2,52.0,2023-12-27
4376,1020.1,8.1,11.4,6.4,53.0,2023-12-28
4377,1016.0,8.8,13.4,4.1,59.0,2023-12-29


In [285]:
spending = pd.read_csv("/Users/satoya01/Lectures/Lecture_2/ds_program/final/code/db/出費.csv")

In [286]:
# 日付を合わせ、出費が0の日を排除
dates_1 = sun_day["年月日"].tolist()
spending["年月日"] = pd.to_datetime(spending["年月日"])

# rows_to_drop = spending[spending["年月日"].isin(dates_1)].index
# spending = spending.drop(rows_to_drop)

spending = spending[spending["年月日"].isin(dates_1)]

spending = spending[spending["出費"] != 0]

dates_2 = spending["年月日"].tolist()

# rows_to_drop = sun_day[sun_day["年月日"].isin(dates_2)].index
# sun_day = sun_day.drop(rows_to_drop)

# sun_day = sun_day[sun_day["年月日"].isin(dates_2)]

spending
# sun_day


,年月日,出費
0,2023-01-01,3470
9,2023-01-10,3690
10,2023-01-11,4070
11,2023-01-12,2680
20,2023-01-21,4180
...,...,...
4369,2023-12-21,4080
4371,2023-12-23,4370
4372,2023-12-24,5270
4373,2023-12-25,6960


In [287]:
sun_day.to_csv("./trash/a.csv", index=False)
spending.to_csv("./trash/b.csv", index=False)


In [288]:
# 年月日を主キーとして spending と sun_day データフレームをマージ
merged_df = pd.merge(sun_day, spending, on='年月日', how='inner')